<a href="https://colab.research.google.com/github/niikun/learning_gemini/blob/main/Function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Function Calling

## 事前準備

In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 6.2 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import google.generativeai as genai
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Automatic Function Calling

### 関数の定義

In [3]:
def add(a:float,b:float) -> float:
    """retrun a+b"""
    return a+b

def sub(a:float,b:float) -> float:
    """retrun a-b"""
    return a-b

def mul(a:float,b:float) -> float:
    """retrun a*b"""
    return a*b

def div(a:float,b:float) -> float:
    """retrun a/b"""
    return a/b if b != 0 else None


In [4]:
model = genai.GenerativeModel(
    model_name="models/gemini-1.5-flash",
    tools=[add,sub,mul,div]
)

In [5]:
chat = model.start_chat(
    enable_automatic_function_calling=True
)

In [6]:
response = chat.send_message(
    "私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?"
 )
response.text

'ミトンは全部で 2508 個あります。'

In [7]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("===")

user -> [{'text': '私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?'}]
===
model -> [{'function_call': {'name': 'mul', 'args': {'a': 57.0, 'b': 44.0}}}]
===
user -> [{'function_response': {'name': 'mul', 'response': {'result': 2508.0}}}]
===
model -> [{'text': 'ミトンは全部で 2508 個あります。'}]
===


## Tool Config  
「 Tool Cong」は 、「 Gemini API」のユーザー入力に応じたメッセージ応答とFunction
Callingの選択を制御する設定です。状況に応じて、使用可能な関数を制限したい場合に
役立ちます。  
次の3つのモードを切り替えることができます。  
- NONE：  メッセージ応答を強制  
- AUTO：メッセージ応答とFunction Callingを自動選択  
- ANY：Function Callingを強制  

#### Tool Config のユーティリティ関数の準備

In [9]:
from google.generativeai.types import content_types
from collections.abc import Iterable

def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Function Calling modeと許可する関数名からTool Configを生成"""
    return content_types.to_tool_config(
        {"function_calling_config" : {"mode":mode, "allowed_function_names":fns}}
    )

#### NONEで動作確認  
NONEを指定すると、必ずメッセージ応答になります。会話履歴から、関数が使われていないことがわかります。

In [11]:
chat.history.clear()

response = chat.send_message(
    "私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?",
    tool_config=tool_config_from_mode("NONE")
)

In [14]:
for content in chat.history:
    # print(content.parts)
    print(content.role, "->",[type(part).to_dict(part) for part in content.parts])
    print("=====")

user -> [{'text': '私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?'}]
=====
model -> [{'text': '猫は57匹で、それぞれの猫が44個のミトンを持っています。ミトンの合計数は、57匹の猫×44個のミトン/猫 = **2508個**です。\n'}]
=====


#### AUTO  
AUTOを指定すると、ユーザー入力に応じてメッセージ応答とFunction Callingを自動選択します。

In [16]:
chat.history.clear()

response = chat.send_message(
    "私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?",
    tool_config=tool_config_from_mode("AUTO")
)

In [18]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])

user -> [{'text': '私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?'}]
model -> [{'function_call': {'name': 'mul', 'args': {'a': 57.0, 'b': 44.0}}}]
user -> [{'function_response': {'name': 'mul', 'response': {'result': 2508.0}}}]
model -> [{'text': '猫は合計2508個のミトンを持っています。 \n'}]


#### ANY
ANYを指定すると、必ずFunction Callingになります。

In [19]:
chat.history.clear()

response = chat.send_message(
    "私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?",
    tool_config=tool_config_from_mode("ANY")
)

In [20]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])

user -> [{'text': '私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?'}]
model -> [{'function_call': {'name': 'mul', 'args': {'a': 57.0, 'b': 44.0}}}]
user -> [{'function_response': {'name': 'mul', 'response': {'result': 2508.0}}}]
model -> [{'text': '猫は合計で 2508 個のミトンを持っています。'}]


In [22]:
chat.history.clear()

response = chat.send_message(
    "明日は晴れますか?",
    tool_config=tool_config_from_mode("ANY")
)

In [23]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])

user -> [{'text': '明日は晴れますか?'}]
model -> [{'text': '申し訳ありませんが、私は天気予報を提供することはできません。'}]


In [24]:
chat.history.clear()

response = chat.send_message(
    "明日は晴れますか?",
    tool_config=tool_config_from_mode("AUTO")
)
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])

user -> [{'text': '明日は晴れますか?'}]
model -> [{'text': '申し訳ありませんが、天気予報を提供することはできません。  私は言語モデルなので、リアルタイムの情報にアクセスできません。'}]


## Manual Function Calling

#### 関数の定義
現在の気温を返す関数を定義します。実運用ではお天気APIから情報を取得する必要が
ありますが、今回は練習用に固定値を返しています。

In [80]:
def get_temperature(location: str):
    """get current temperature."""
    if location == "東京":
        return "20度"
    else:
        return "10度"

In [81]:
functions = {
 "get_temperature": get_temperature,
 }

In [82]:
model = genai.GenerativeModel(
    model_name="models/gemini-1.5-flash",
    tools=functions.values(),
 )

In [83]:
model = genai.GenerativeModel(
    model_name="models/gemini-1.5-flash",
    tools=functions.values(),
 )

In [101]:
response = model.generate_content(
    "how temperature in tokyo?"
)
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_temperature",
                  "args": {
                    "location": "tokyo"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_SEXUALLY_E

In [56]:
import requests

In [63]:
def get_temperature():
    """get tomorrow temperature"""
    URL = "https://api.open-meteo.com/v1/forecast?latitude=35.6895&longitude=139.6917&daily=weather_code,temperature_2m_max"
    res = requests.get(URL)
    res.json()
    temperature_tommorow = res.json()["daily"]["temperature_2m_max"][1]
    return temperature_tommorow

def get_weather():
    """get tommorow weather"""
    URL = "https://api.open-meteo.com/v1/forecast?latitude=35.6895&longitude=139.6917&daily=weather_code,temperature_2m_max"
    res = requests.get(URL)
    res.json()
    weather_tommorow_code = res.json()["daily"]["weather_code"][1]
    if weather_tommorow_code <= 1:
        weather_tommorow = "晴れ"
    elif weather_tommorow_code <= 3:
        weather_tommorow = "曇り"
    else:
        weather_tommorow = "雨"
    return weather_tommorow

In [64]:
get_temperature(),get_weather()

(36.6, '曇り')